# 6.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле. 

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.


In [28]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class sample(object):
  def __init__(self, X, n_subspace):
    self.idx_subspace = self.random_subspace(X, n_subspace)
  
  def __call__(self, X, y):
    idx_obj = self.bootstrap_sample(X)
    X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
    return X_sampled, y_sampled

  @staticmethod
  def bootstrap_sample(X, random_state=42):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи бэггинга индексов.
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    return np.unique(np.random.choice(len(X), len(X)))
  @staticmethod
  def random_subspace(X, n_subspace, random_state=42):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи метода случайных подпространств признаков
    Количество этих признаков передается при помощи аргумента n_subspace
    Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
    """
    return np.random.choice(len(X[0]), n_subspace, replace=False)

  @staticmethod
  def get_subsample(X, y, idx_subspace, idx_obj):
    """
    Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled 
    по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
    """
    x_sampled=X[np.ix_(idx_obj, idx_subspace)]
    y_sampled=y[np.array(idx_obj)]
    return x_sampled,y_sampled



class random_forest(object):
  def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.subspaces_dim = subspaces_dim
    self.random_state = random_state
    self.trees=[]
    self.subspace_idx=[]

  def fit(self, X, y):
    for i in range(self.n_estimators):
      s = sample(X, self.subspaces_dim)
      bootstrap_indices = s.bootstrap_sample(X)
      X_sampled, y_sampled = s.get_subsample(X, y, s.idx_subspace, bootstrap_indices)
      self.subspace_idx.append(s.idx_subspace)
      clf = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
      clf.fit(X_sampled, y_sampled)
      self.trees.append(clf)
    self.trees=np.array(self.trees)
    self.subspace_idx=np.array(self.subspace_idx)
  def predict(self, X):
    ans=[]
    for x in X :
      dict=[]
      x=np.array(x)
      x=x[np.newaxis, :]
      for i in range(self.n_estimators):
        dict.append(self.trees[i].predict(x[np.ix_([0], self.subspace_idx[i])]))
      values, counts = np.unique(np.array(dict), return_counts=True)
      ans.append(values[np.argmax(counts)])
    return np.array(ans)
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.3, stratify=data.target, random_state=42)

N_ESTIMATORS=[1, 2, 3, 4,  5, 6, 7 ,8, 9 ,10, 11, 12, 13, 14,15,  16, 17, 19]#, 22, 23, 24,  25, 26, 27, 28, 31, 35, 40 , 45, 46, 46, 48, 49, 50]
MAX_DEPTH=[1, 2, 3, 4,  5, 6, 7 ,8, 9 ,10, 11, 12, 13, 14,15,  16, 17, 19]#, 22, 23, 24,  25, 26, 27, 28, 31, 35, 40 , 45, 46, 46, 48, 49, 50]
SUBSPACE_DIM=[1, 2, 3, 4]
max=-1
for i in tqdm(N_ESTIMATORS):
  for j in MAX_DEPTH:
    for k in SUBSPACE_DIM:
      rf=random_forest(n_estimators=i, max_depth=j, subspaces_dim=k, random_state=42)
      rf.fit(X_train, y_train)
      ans=rf.predict(X_test)
      #print(len(ans), len(y_test))
      acc=accuracy_score(y_test, ans)
      if acc>max :
        max=acc
        max_i=i
        max_j=j
        max_k=k
print(max_i, max_j, max_k)





100%|██████████| 18/18 [00:58<00:00,  3.22s/it]

1 6 3
